In [2]:
#Inizio parte sui classificatori

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
# import seaborn as sns
import plotly.graph_objects as go
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

Importiamo il DataFrame con i dati necessari sui consumi, poi specializziamo nell'area urbana di Trento.

In [ ]:
df_l = pd.read_csv('dataframe/dati_orari_medie_provincia_Trento.csv')
#cambiare nome file

In [ ]:
df_l = df_l[df_l['COMUNE']=='TRENTO']

Per il preprocessing dei dati, uso come riferimento del filtro la classe di consumo 'alta'

In [ ]:
filter = [df_l['fascia_oraria']=='8-12',df_l['fascia_oraria']=='12-14',df_l['fascia_oraria']=='14-19',(df_l['fascia_oraria']!='19-00')&(df_l['fascia_oraria']!='00-08')]

df = df_l[filter[2]] #ok ma devo scegliere una fascia oraria da analizzare
target_class = 'alto'

Qui ci troviamo di fronte a due possibili scelte di insiemi di training e test. Tenendo conto della sequenza temporale, e quindi volendo analizzare i consumi futuri, possiamo usare come training il mese di novembre e la prima metà di dicembre, per fare il test sulla seconda metà di dicembre. Siccome tale scelta può produrre discrepanze viziate dalle vacanze natalizie, la seconda scelta è una divisione casuale (80%-20%) dei giorni, in modo da avere dei risultati più mediati, in positivo e/o in negativo.

Qui riporto la prima scelta, quella basata su criteri temporali.

In [ ]:
# Divido il dataset in training e test (prima e dopo il 15 dicembre)
X_train = df[df['data'] < '2013-12-15'].drop(columns = {'consumo','data','corrente_pesata','COMUNE'})
# Creo un array con gli identificativi delle celle del training
cell_train = X_train['cellId'].unique()
X_test = df.query("data >= '2013-12-15' and cellId in @cell_train").drop(columns = {'consumo', 'data', 'corrente_pesata', 'COMUNE'})
y_train = df[df['data'] < '2013-12-15']['consumo']
y_train = (y_train == target_class).astype(int)
y_test = df.query("data >= '2013-12-15' and cellId in @cell_train")['consumo']
y_test = (y_test == target_class).astype(int)

Adesso riporto la seconda opzione, quella di divisione casuale.

In [ ]:
X = df.drop(columns= {'consumo','data','corrente_pesata','COMUNE'})
y = df['consumo']
y_bin = (y == target_class).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=0.2, random_state=24)

Predispongo la pipeline per un lavoro preliminare sui dati. Per quanto riguarda i dati mancanti, abbiamo agito in due modi al variare del tipo di feature (numerica o di categoria). Per la prima, sostituisco il dato problematico con una media e poi normalizzo tutte le features. Per la seconda, invece, sostituisco con valore 'missing', in quanto pensiamo che questa nuova categoria non influenzi particolarmente l'analisi complessiva.

In [ ]:
numeric_features = ['temp','prec','tweet','pioggia','mod_wind','ang_wind']
categorical_features = ['cellId','fascia_oraria','day_week']

#Per le features numeriche, uso la media in caso di dati mancanti, poi normalizzo tutto
numeric_transformer = make_pipeline(SimpleImputer(strategy='mean'), 
                                    StandardScaler())

#Per le features di categoria, sostituisco le celle mancanti con 'missing' values, mentre ignoro successivamente i valori non visti durante il training
categorical_transformer = make_pipeline(SimpleImputer(strategy='constant', fill_value='missing'), 
                                        OneHotEncoder(handle_unknown='ignore', sparse_output = False))
       
#Unisco le due pipelines precedenti in un'unica struttura
preprocessor = make_column_transformer(
        (numeric_transformer, numeric_features),
        (categorical_transformer, categorical_features)
)

In [ ]:
# Creazione della pipeline finale
pipeline = make_pipeline(preprocessor)

model_rf = make_pipeline(preprocessor, RandomForestClassifier(random_state=24))

model_lr = make_pipeline(preprocessor, LogisticRegression(random_state=24))

Adesso procediamo con il modello di random forest, fare delle predizioni e valutarne i risultati.
Costruiamo quindi i dati per la matrice di confusione, la curva ROC e il grafico per la precision-recall.

In [ ]:
#RANDOM FOREST
#Addestro sui dati di training
model_rf.fit(X_train, y_train)

#Predizione sui dati di test
y_pred = model_rf.predict(X_test)
#Probabilità per la classe positiva
y_prob = model_rf.predict_proba(X_test)[:, 1]

In [ ]:
#Valutazioni del modello
report_rf = classification_report(y_test, y_pred)
print(report_rf)

#Costruisco e mostro confusion matrix
confusion_matrix_rf = confusion_matrix(y_test.values,model_rf.predict(X_test)) #uso .values per avere solo l'array associato
confusion_matrix_disp_rf = ConfusionMatrixDisplay(confusion_matrix_rf)

#Calcolo ROC curve e AUC area associata
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_prob)
auc_rf = auc(fpr_rf, tpr_rf)

#Calcolo precision recall curve e soglie usate
pr_rf, rec_rf, thresh_rf = precision_recall_curve(y_test, y_prob)


Questa volta operiamo con il logistic regressor, come sopra.

In [ ]:
#LOGISTIC REGR
#Addestro sui dati di training
model_lr.fit(X_train, y_train)

#Predizione sui dati di test
y_pred = model_lr.predict(X_test)
#Probabilità per la classe positiva
y_prob = model_lr.predict_proba(X_test)[:, 1]

In [ ]:
#Valutazioni del modello
report_lr = classification_report(y_test, y_pred)
print(report_lr)

#Costruisco e mostro confusion matrix
confusion_matrix_lr = confusion_matrix(y_test.values,model_lr.predict(X_test)) #uso .values per avere solo l'array associato
confusion_matrix_disp_lr = ConfusionMatrixDisplay(confusion_matrix_lr)

#Calcolo ROC curve e AUC area associata
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, y_prob)
auc_lr = auc(fpr_lr, tpr_lr)

#Calcolo precision recall curve e soglie usate
pr_lr, rec_lr, thresh_lr = precision_recall_curve(y_test, y_prob)


Plotto le matrici di confusione, le ROC curves e le precision-recall curves per Random Forest e Logistic Regressor, in una griglia 3x2 in modo da poter comprare facilmente le differenze fra i due modelli e le validità di entrambi.

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(14, 18))

#Plot per matrici di confusione
confusion_matrix_disp_rf.plot(ax=axes[0, 0], colorbar=False)
axes[0, 0].set_title('Confusion Matrix Random Forest')
confusion_matrix_disp_lr.plot(ax=axes[0, 1], colorbar=False)
axes[0, 1].set_title('Confusion Matrix Logistic Regressor')

#Plot per curve ROC
#rf
axes[1, 0].plot(fpr_rf, tpr_rf, color='blue', lw=2, label=f'ROC curve (area = {auc_rf:.2f})')
axes[1, 0].plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
axes[1, 0].set_xlim([0.0, 1.0])
axes[1, 0].set_ylim([0.0, 1.05])
axes[1, 0].set_xlabel('False Positive Rate')
axes[1, 0].set_ylabel('True Positive Rate')
axes[1, 0].set_title('ROC Random Forest')
axes[1, 0].legend(loc="lower right")
#lr
axes[1, 1].plot(fpr_lr, tpr_lr, color='blue', lw=2, label=f'ROC curve (area = {auc_lr:.2f})')
axes[1, 1].plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
axes[1, 1].set_xlim([0.0, 1.0])
axes[1, 1].set_ylim([0.0, 1.05])
axes[1, 1].set_xlabel('False Positive Rate')
axes[1, 1].set_ylabel('True Positive Rate')
axes[1, 1].set_title('ROC Logistic Regressor')
axes[1, 1].legend(loc="lower right")

#Plot per precision-recall curve
#rf
axes[2, 0].plot(rec_rf, pr_rf, color='green', lw=2, label='Precision-Recall curve')
axes[2, 0].set_xlim([0.0, 1.0])
axes[2, 0].set_ylim([0.0, 1.05])
axes[2, 0].set_xlabel('Recall')
axes[2, 0].set_ylabel('Precision')
axes[2, 0].set_title('Precision-Recall Random Forest')
axes[2, 0].legend(loc="lower left")
#lr
axes[2, 1].plot(rec1, pr1, color='green', lw=2, label='Precision-Recall curve')
axes[2, 1].set_xlim([0.0, 1.0])
axes[2, 1].set_ylim([0.0, 1.05])
axes[2, 1].set_xlabel('Recall')
axes[2, 1].set_ylabel('Precision')
axes[2, 1].set_title('Precision-Recall Logistic Regressor')
axes[2, 1].legend(loc="lower left")

#Mostro infine il plot con un layout pulito
plt.tight_layout()
plt.show()
